# Generate the Input Table for the Format App
## 1. Parameters & Libs

In [1]:
import pandas as pd
import numpy as np
import gpc_utils.emblem as eu
import gpc_utils.sas as su
from tqdm import tqdm
import pickle
import os
import pyarrow.parquet as pq
from src.formats import define_format, parse_format_dict_row
from src.univariate import univariate_plotly

import warnings
# Ignore FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
path_df = r"./data/raw_data.prq"
path_formats_table = r"./data/formats.xlsx"
figures_dict_path = r"./data/figures_univariates.pkl"
tables_dict_path = r"./data/tables_univariates.pkl"

path_data_dict = r"./metadata/data_dict.xlsx"
path_model_dict = r"./metadata/model_dict.xlsx"
path_formats_dict = r"./metadata/formats_dict.xlsx"

## 2. Load Metadata

In [3]:
data_dict = pd.read_excel(path_data_dict, "2025_all")
data_dict = data_dict[data_dict[f"facbid_DAG"]==1]
data_dict.head()

,Factores,Modelo Anterior,LABEL,CATEGORÍA,NOTAS,DAG,CRI,ROB,INC,FEL,...,BASE_FORZADA,BASE,MISSING,OTHER,Type,missing_level,other_level,is_categorical,monotonicity,vbles_xsub
1,ALARMA,SI,Posee o no alarma,03.Riesgo,NaN,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,1.0,1.0,policy,Missing,Other,NaN,NaN,1
2,ANTIGUEDAD_POLIZA,NaN,Antiguedad de la Poliza,00.Control,NaN,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,1.0,1.0,policy,Missing,Other,1.0,policy,1
5,ANTIGVIV,NO,Antiguedad de la vivienda,03.Riesgo,NaN,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,1.0,1.0,NaN,Missing,Other,NaN,NaN,1
6,ANUALIDAD_REEMPLAZO,SI,Anualidad del remplazo,00.Control,NaN,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,1.0,1.0,policy,Missing,Other,0.0,0,1
7,ANUALIDAD_SEGURO,SI,Anualidad del Seguro,00.Control,NaN,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,1.0,1.0,policy,Missing,Other,0.0,0,1


In [4]:
formats_table = pd.read_excel(path_formats_table)
formatted_vars = [x.replace("FMT_", "") for x in formats_table.FMTNAME.unique()]
unformatted_vars = [x for x in data_dict["Factores"] if x not in formatted_vars]

formats_table = formats_table.drop_duplicates()

formats_table

,START,END,FMTNAME,TYPE,LABEL,HLO,SEXCL,EEXCL
0,LOW,1903,FMT_ANYOCONS_EDIF,N,<1903,SL,Y,Y
1,1903,1914,FMT_ANYOCONS_EDIF,N,"[1903 , 1914]",S,N,N
2,1914,1916,FMT_ANYOCONS_EDIF,N,"(1914 , 1916]",S,Y,N
3,1916,1919,FMT_ANYOCONS_EDIF,N,"(1916 , 1919]",S,Y,N
4,1919,1921,FMT_ANYOCONS_EDIF,N,"(1919 , 1921]",S,Y,N
...,...,...,...,...,...,...,...,...
12621,87,HIGH,FMT_EU_TOPO_CONV_24Q3,N,>87.00,SH,Y,Y
12622,.,.,FMT_EU_TOPO_CONV_24Q3,N,Missing,S,N,N
12623,***OTHER**,***OTHER**,FMT_EU_TOPO_CONV_24Q3,N,Other,SO,N,N
12624,-9999,-9999,FMT_EU_TOPO_CONV_24Q3,N,Missing,S,N,N


## 3. Apply Formats

In [5]:
formatted_vars = [x for x in formatted_vars if (x in data_dict.Factores.to_list() and x != "ANTIGUEDAD_POLIZA")]

In [11]:
weight = "EXP_CORR_GAGUA_CTE"
resp = "STRO_CORR_DAGCTSE"

pq_file = pq.ParquetFile(path_df)

figures_dict = {}
tables_dict = {}

for var in tqdm(formatted_vars):

    if var not in figures_dict:

        # get data in memory from the spark connection
        df_var = pq_file.read(columns=[var, resp, weight], use_pandas_metadata=True).to_pandas()
        
        # format the in-memory dataframe and generate a plot
        df_var[var+"_formatted"] = su.apply_format(vec=df_var[var], fmt_table=formats_table[formats_table.FMTNAME=="FMT_"+var])
        table, fig = univariate_plotly(df_var, x=var+"_formatted", y=resp, fig_title=data_dict[data_dict.Factores==var]["LABEL"].item(),
                                       w=weight,fig_w=1100, fig_h=700, retfig=True, show_fig=False, output=True)

        # store format table and plot
        figures_dict[var] = fig
        tables_dict[var] = table[var+'_formatted'][0]

with open(figures_dict_path, "wb") as file:
    pickle.dump(figures_dict, file)

with open(tables_dict_path, "wb") as file:
    pickle.dump(tables_dict, file)

  0%|          | 0/337 [00:00<?, ?it/s]

100%|██████████| 337/337 [00:17<00:00, 19.25it/s]
